In [1]:
import simulation as sim
from importlib import reload
import numpy as np
import pickle
import random
from joblib import Parallel, delayed
from tqdm import tqdm
import pandas as pd
import os
from scipy import stats
from scipy.stats import skewnorm
from scipy.optimize import minimize_scalar
from scipy.stats import skewnorm
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
#trials = pd.read_pickle('BEL_trials_14_Optimising_with_initial_probing_lumenvolcost_fccstart_pos.pkl')
#trials = pd.read_pickle('BEL_trials_15_Optimising_with_initial_probing_lumenvolcost_fccstart_pos_goodrangeonly.pkl')


In [2]:
def instantiate_and_run_simulation(
    run_number
):
    simulation = sim.Simulation(N_bodies=70)

    beta = random.uniform(1,200) # this was initially optimised to 1-> 400
    alpha = 0.3 # this was initially optimised to 0.000078 -> 0.5
    A_eq_star_scaling = random.uniform(0.000111,0.4)
    P_star = 1 # this was initially optimised to 0.1 -> 1.1
    radius_scaling = random.uniform(0.5, 1.8)
    volume_scaling = 0.1 #in initial optimisiation this was 0.01 -> 0.1

    simulation.execute(
        beta=beta,
        alpha=alpha,
        A_eq_star_scaling=A_eq_star_scaling,
        P_star=P_star,
        radius_scaling=radius_scaling,
        volume_scaling = volume_scaling,
        write_results=True,
        # max_reset_count=20,
        run_number=run_number,
        write_path="f:\\Bel_Simulation\\Outputs_no_optimisation_good_range_70init_0p1vol_Pstar_alphafixed_1in20_std"
        )

    return simulation

In [4]:
# simulation = instantiate_and_run_simulation(1)

In [4]:
with sim.tqdm_joblib(tqdm(desc="My Simulation", total=1_000_000)) as progress_bar:
    Parallel(n_jobs=5)(delayed(instantiate_and_run_simulation)(j+2313) for j in range(1_000_000))

My Simulation:   0%|          | 566/1000000 [6:01:14<10631:08:42, 38.29s/it] 


BrokenProcessPool: A result has failed to un-serialize. Please ensure that the objects returned by the function are always picklable.

In [7]:
def scaled_gaussian(x, mu, sigma):
    max_val = 1 / (sigma * np.sqrt(2 * np.pi))
    pdf_val = (1 / (sigma * np.sqrt(2 * np.pi))) * np.exp(-0.5 * ((x - mu) / sigma) ** 2)
    y = pdf_val / max_val
    return y

class ScaledSkewNormal:

    def __init__(self, a, mu, sigma):
        neg_pdf = lambda x: -skewnorm.pdf(x, a, mu, sigma)
        self.result = minimize_scalar(neg_pdf, bounds=(mu - 3*sigma, mu + 3*sigma), method='bounded')
        self.pdf_max_value = skewnorm.pdf(self.result.x, a, mu, sigma)
        self.a = a
        self.mu = mu
        self.sigma = sigma

    def get_value(self, x):
        pdf_val = skewnorm.pdf(x, self.a, self.mu, self.sigma)
        return pdf_val / self.pdf_max_value

def lumen_vol_dist(x):
    if 0 <= x <= 0.5:
        return 1
    elif 0.5 < x <= 1:
        return -2 * x + 2
    else:
        return 1e-30

def time_dist(x):
    return (x / 15) + 1e-30

def lumen_com_dist(x):
    return 1 / np.exp(0.5*x)

def derive_target(
    mean_separation: float,
    lumen_com: float,
    sphericity: float,
    n_cells: float,
    lumen_vol: float,
    hull_vol: float,
    max_time: float
) -> float:
    """
    Derive the value to be maximised from the results of the simulation.
    
    Inputs:
        results: The results of the simulation.
    
    Returns:
        The value to be maximised.
    """

    #skew_normal_volume = ScaledSkewNormal(5, -50, 500)
    skew_normal_n_cells = ScaledSkewNormal(5, -80, 500)
    skew_normal_sphericity = ScaledSkewNormal(-5, 1.05, 0.3)
    
    lumen_com_optimisation_values = lumen_com_dist(lumen_com)
    mean_separation_optimisation_value = scaled_gaussian(mean_separation, mu=-0.2, sigma=0.3)
    #volume_optimisation_value = skew_normal_n_cells.get_value(volume)
    sphericity_optimisation_value = skew_normal_sphericity.get_value(sphericity)
    n_cells_optimisation_value = skew_normal_n_cells.get_value(n_cells)
    lumen_vol_optimisation_value = lumen_vol_dist(lumen_vol/hull_vol)
    time_optimisation_value = time_dist(max_time)

    target_value = (
        mean_separation_optimisation_value *
        #volume_optimisation_value *
        sphericity_optimisation_value *
        n_cells_optimisation_value *
        lumen_vol_optimisation_value *
        time_optimisation_value *
        lumen_com_optimisation_values
    )

    return target_value

In [10]:
simulation = instantiate_and_run_simulation(1)

In [11]:
results = simulation.results.iloc[-1]
target = -derive_target(
    results['mean_separation'],
    results['lumen_distance_from_com'],
    results['sphericity'],
    results['final_N_bodies'],
    results['lumen_volume'],
    results['hull_volume'],
    results['t']
)

In [14]:
target

-0.6481651365106207

In [ ]:
results = pd.concat([pd.read_parquet(f"C:\\Users\\Tom\\Documents\\Bel PhD\\Bel_Simulation\\outputs\\{i}") for i in os.listdir("C:\\Users\\Tom\\Documents\\Bel PhD\\Bel_Simulation\\outputs") if i.endswith(".parquet")])

In [10]:
simulation = sim.Simulation(N_bodies=50, timestep_reset=1000)

In [11]:
np.linalg.norm(np.mean(simulation.positions, axis=0)-simulation.positions[-1])

0.002725328356610611

In [18]:
np.linalg.norm(np.mean(simulation.positions, axis=0)- simulation.positions[-1])

0.002725328356610611